In [1]:
pip install --upgrade pip

  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
      Successfully uninstalled pip-21.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [63]:
import numpy as np 
import cv2 
import matplotlib.pyplot as plt
import PIL.Image as Image
import gym
import random
import math

from gym import Env, spaces
import time

font = cv2.FONT_HERSHEY_COMPLEX_SMALL 

In [74]:
class tree(object):
    def __init__(self, name, x,y, b):
        self.x = x
        self.y = y
        self.name = name
        self.icon = cv2.imread("tree.png") / 255.0
        self.icon_w = 70
        self.icon_h = 70
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))
        self.berries = b
        
    def reduce(self):
        self.berries *= 0.9
        return self.berries
    
    def replenish(self):
        self.berries += 4
        self.berries = min(self.berries, 200)

In [75]:
class man(object):
    def __init__(self, name, x,y):
        self.x = x
        self.y = y
        self.name = name
        self.icon = cv2.imread("man.png") / 255.0
        self.icon_w = 50
        self.icon_h = 50
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))
        self.berries = 0
        
    def move(self, new_x, new_y):
        self.x = new_x
        self.y = new_y
        
    def eat(self, b):
        self.berries += b

In [116]:
class berryPatches(Env):
    def __init__(self):
        super(berryPatches, self).__init__()
        
        # Define a 2-D observation space
        self.canvas_shape = (800, 800, 3)
        # 9 places: 8 trees + 1 center
        self.observation_space = spaces.Discrete(9)
    
        
        # Define an action space: 8 tree travels + harvest
        self.action_space = spaces.Discrete(9)
                        
        # Create a canvas to render the environment images upon 
        self.canvas = np.ones(self.canvas_shape) * 1
        
        # Define elements present inside the environment
        self.elements = []
        self.trees =[]
        self.man =man("man",350,300)
        
        # Maximum time 
        self.max_time = 300

        # Position of man 
        self.man_tree = self.man
    
    def get_action_meanings(self):
        return {0: "harvest", 
                1: "patch 1 (3 o'clock)", 
                2: "patch 2", 
                3: "patch 3", 
                4: "patch 4", 
                5: "patch 5", 
                6: "patch 6", 
                7: "patch 7", 
                8: "patch 8", 
               }

        
    def step(self, action):
        # Flag that marks the termination of an episode
        done = False

        # Assert that it is a valid action 
        assert (self.action_space.contains(action) and (self.time_left!=0 or action!=0)) , "Invalid Action"

        # apply the action 
        if action == 0:
            self.time_left -= 1
            reward = self.man_tree.reduce()
            self.man.eat(reward)
            if self.man_tree.berries!=0:
                for t in self.trees:
                    if(t==self.man_tree or t.berries ==0): continue
                    t.replenish()
        elif action in [8,1,2]:
            self.man.move(self.trees[action-1].x +70 ,self.trees[action-1].y)
        elif action in [4,5,6]:
            self.man.move(self.trees[action-1].x -60 ,self.trees[action-1].y)
        elif action == 7:
            self.man.move(self.trees[action-1].x ,self.trees[action-1].y - 70)
        else:
            self.man.move(self.trees[action-1].x ,self.trees[action-1].y + 83)
        if action !=0 :
            reward =0
            self.time_left -= math.sqrt((self.man_tree.x - self.trees[action-1].x)**2 +
                                        (self.man_tree.y - self.trees[action-1].y)**2)/150
            self.man_tree=self.trees[action-1]
        
        # Increment the episodic return
        self.ep_return += reward
        
        # Draw elements on the canvas
        self.draw_elements_on_canvas()

        # If out of time, end the episode.
        if self.time_left == 0:
            done = True

        return self.canvas, reward, done, []

    def reset(self):
        # Reset the time consumed
        self.time_left = self.max_time

        # Reset the reward
        self.ep_return  = 0

        # Intialise the elements 
        self.elements = []
        self.trees=[]
        berries =[0,70,70,0,70,0,70,0]
        x = 200
        y = 0
        for i in range(8):
            # Intialise the patches
            tree1 = tree("tree", int(350 + x), int(300+y),berries[i])
            self.elements.append(tree1)
            x= 200*math.cos((i+1)*math.pi/4) 
            y= 200*math.sin((i+1)*math.pi/4) 
        
        self.trees = list(self.elements)
        self.man =man("man",350,300)
        self.elements.append(self.man)
        self.man_tree= self.man
        
        # Reset the Canvas 
        self.canvas = np.ones(self.canvas_shape) * 1

        # Draw elements on the canvas
        self.draw_elements_on_canvas()


        # return the observation
        return self.canvas 

    def draw_elements_on_canvas(self):
        
        # Init the canvas 
        self.canvas = np.ones(self.canvas_shape) * 1

        # Draw the patches on canvas
        for elem in self.elements:
            elem_shape = elem.icon.shape
            x,y = elem.x, elem.y
            self.canvas[y : y + elem_shape[1], x:x + elem_shape[0]] = elem.icon
            text = str(round(elem.berries,2))
            self.canvas = cv2.putText(self.canvas, text, (x+int(elem_shape[0]/2.5),y+elem_shape[1] +10), font,  
                  0.6, (0,0,0), 1, cv2.LINE_AA)

        text = 'Time Left: {:.2f} | Rewards: {}'.format(self.time_left, self.ep_return)

        # Put the info on canvas 
        self.canvas = cv2.putText(self.canvas, text, (10,20), font,  
                  0.8, (0,0,0), 1, cv2.LINE_AA)

        
    def render(self, mode = "human"):
        assert mode in ["human", "rgb_array"], "Invalid mode, must be either \"human\" or \"rgb_array\""
        if mode == "human":
            cv2.imshow("Game", self.canvas)
            cv2.waitKey(4000)

        elif mode == "rgb_array":
            return self.canvas
    
    def close(self):
        cv2.destroyAllWindows()
        
        

In [117]:
env = berryPatches()
obs = env.reset()


In [118]:
for i in range(8):
    action = 1+i
    obs, reward, done, info = env.step(action)
    env.render()
    obs, reward, done, info = env.step(0)
    env.render()
    if done == True:
        break

env.close()
